In [4]:
using JuMP, Gurobi, DelimitedFiles, Printf;

###################################
# GLOBAL VARIABLES
const GUROBI_ENV = Gurobi.Env()
###################################

Set parameter Username
Academic license - for non-commercial use only - expires 2023-03-09


Gurobi.Env(Ptr{Nothing} @0x0000000040ac6d20, false, 0)

## LoadInstance

- Recuerde tener el archivo instancia.txt en la misma carpeta de este notebook.
- NO modifique nada de la función loadInstance(). 
- Como puede ver, la funcion f viene dentro de la instancia, y la funcion de costos se calcula como la distancia euclideana entre los puntos. Los vertices estan enumerados de 1 hasta n.

In [5]:
function loadInstance(fn::AbstractString)
  W = readdlm(fn, ',', Int64, '\n');

  f = W[:,1]
  x = W[:,2]
  y = W[:,3]
  n = length(f);
  
  E = [];
  c = [];
  for u in 1:n
    for v in u+1:n
      append!(E, [vcat(u,v)]);
      append!(c,((x[u]-x[v])^2+(y[u]-y[v])^2)^0.5);
    end
  end

  return E, c, 1:n, f;
end

loadInstance (generic function with 1 method)

## Importar datos

Solo ejecute la celda de abajo. Esta función le dará

- E: Conjunto de aristas.
- c: funcion de costos de cada arista. El costo de la arista E[i] es c[i].
- N: Conjunto de vertices {1,...,N}.
- Funcion de ganancia de cada nodo.

In [6]:
# Importar Datos
fn = "instancia";
E, c, N, f = loadInstance(fn * ".txt");

In [7]:
print(f)

[25, 24, 26, 20, 30, 24, 22, 27, 21, 28, 28, 21, 27, 21, 28, 30, 23, 29, 30, 25, 27, 22, 27, 25, 21, 28, 22, 24, 28, 22, 22, 24, 30, 24, 25, 27, 23, 22, 30, 28]

In [8]:
#Funciopn que calcula el corte de un vertice
function corte(v,E)
    lista=[]
    largo=length(E)
    i=1
    while i <= largo
        if v in E[i]
            push!(lista,i)
        end
        i=i+1
    end
    return E[lista]
end

corte (generic function with 1 method)

In [9]:
corte_aux=corte(3,E)
[1,3] in corte_aux

true

## Resuelva relajacion de PCTSP

In [10]:
#Resuelva la relajacion lineal de TCTSP

mdl = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV),"Presolve"=>0, "Cuts"=> 0, "Heuristics"=> 0, "Threads"=>1))
    
set_silent(mdl)


#----------------------------
#     INSERTE SU CODIGO AQUI

#Variables

x = @variable(mdl,x[1:length(E)])
y = @variable(mdl,y[N])
#print(x)
#print(y)
#delta_E=

#Restricciones
#Restricciones para que vivan en [0,1]
for i in 1:length(E)
    @constraint(mdl,x[i]<=1)
    @constraint(mdl,-x[i]<=0)
end
for i in N
    @constraint(mdl,y[i]<=1)
    @constraint(mdl,-y[i]<=0)
end
#restriccion de grado
for i in N
    corte_aux=corte(i ,E)
    @constraint(mdl,sum(x[w] for w in 1:length(E) if  E[w] in corte_aux )==2*y[i])
#Restriccion origen
end
@constraint(mdl,y[1]==1)

#Funcon Objetivo
@objective(mdl,Max,sum(f[v]y[v] for v in N)-sum(c[e]x[e] for e in 1:length(E)))

optimize!(mdl)
has_values(mdl)
xv = value.(mdl[:x])
yv = value.(mdl[:y])

#----------------------------

Set parameter Presolve to value 0
Set parameter Cuts to value 0
Set parameter Heuristics to value 0
Set parameter Threads to value 1


1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:40
And data, a 40-element Vector{Float64}:
 1.0
 1.0
 1.0
 0.5
 1.0
 0.0
 0.5
 0.0
 1.0
 1.0
 1.0
 0.0
 1.0
 ⋮
 1.0
 0.5
 1.0
 1.0
 1.0
 1.0
 1.0
 0.5
 0.5
 1.0
 1.0
 1.0

In [105]:
objective_value(mdl)

378.7151557163601

In [127]:
function indices_corte(v,E)
    lista=[]
    largo=length(E)
    i=1
    while i <= largo
        if v in E[i]
            push!(lista,i)
        end
        i=i+1
    end
    return lista
end
indices_corte(2,E)

39-element Vector{Any}:
  1
 40
 41
 42
 43
 44
 45
 46
 47
 48
 49
 50
 51
  ⋮
 66
 67
 68
 69
 70
 71
 72
 73
 74
 75
 76
 77

In [135]:
    arista_corte_aux=indices_corte(1,E)
    E_=[e for  e in 1:length(E)if  !(e   in arista_corte_aux) ]

741-element Vector{Int64}:
  40
  41
  42
  43
  44
  45
  46
  47
  48
  49
  50
  51
  52
   ⋮
 769
 770
 771
 772
 773
 774
 775
 776
 777
 778
 779
 780

## Itere 5 veces resolviendo PSep, y añadiendo la restriccion GSEC correspondiente

$\textbf{IMPORTANTE}$: Debe mantener todas las restricciones que vaya incorporando. Es decir, en la primera iteracion hay una restriccion GSEC, en la segunda hay dos restricciones GSEC,..., en la 5ta hay 5 restricciones GSEC (al volver a resolver la relajación de PCTSP).

Si en alguna iteracion no se puede agregar ninguna restricción GSEC adicional, termine el loop.

In [191]:
for i in 1:5
    
    
   #Defina modelo PSep relajado 
   mdl2 = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV),"Presolve"=>0, "Cuts"=> 0, "Heuristics"=> 0, "Threads"=>1))
    
    set_silent(mdl2)
    
    #------------------------------------------
    
    #     INSERTE SU CODIGO AQUI
    
    #Variables
    arista_corte_aux=indices_corte(1,E)
    Ez=[e for  e in 1:length(E)if  !(e   in arista_corte_aux) ]
    w = @variable(mdl2,w[Ez])
    Zz=[i for i in N if i!=1]
    z = @variable(mdl2,z[Zz])
    
    #Restricciones
   for i in Ez
        @constraint(mdl2,w[i]<=z[E[i][1]])
        @constraint(mdl2,w[i]<=z[E[i][2]])

        @constraint(mdl2,w[i]>=z[E[i][1]]+z[E[i][2]] -1) 
        @constraint(mdl2,w[i]<=1)
        @constraint(mdl2,-w[i]<=0)
    end
    for i in Zz
        @constraint(mdl2,z[i]<=1)
        @constraint(mdl2,-z[i]<=0)
    end 
    #Buscamos que t funciona
    t=40
    V_=[i for i in 2:40 if i!=t]
    print(V_)
    @constraint(mdl2,z[t]==1)

    #Funcion objetivo
    @objective(mdl2,Max,sum(xv[e]w[e] for e in Ez)-sum(yv[e]z[e] for e in V_))
    
   
    
    optimize!(mdl2)
    print(has_values(mdl2))
    vpsep = objective_value(mdl2)
    println(vpsep)
    wv =  value.(mdl2[:w])
    zv =  value.(mdl2[:z])
    print(zv)
    
   # if  objective_value(mdl2)>0
    #    S =[i for i in Zz if( zv[]]
    
     #-----------------------------------------
    
    println("El valor optimo de PSep es {}".format(vpsep))
    
    print("Nueva restriccion por añadir es S=")
    println(S)
    
    
    
    #Vuelva a resolver PCTSP relajado, añadiendo la restriccion GSEC encontrada arriba
    
    mdl = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV),"Presolve"=>0, "Cuts"=> 0, "Heuristics"=> 0, "Threads"=>1))
    
    set_silent(mdl)
    
    #----------------------------
    #     INSERTE SU CODIGO AQUI

  #  x = @variable(...)
   # y = @variable(...)
    
    #Retricciones
    
    #Funcion objetivo
   

    optimize!(mdl)
    
    tcv = #Obtenga el valor del optimo
    xv = #Obtenga solucion optima de x
    yv = #Obtenga solucion optima de y
     #----------------------------
    
    #Imprima el valor valor del problema, y el valor de x,y
    
    print("El valor obtenido es:")
    println(tcv)
    
    print("El valor de x obtenido es:")
    println(xv)
    
    
    print("El valor de y obtenido es:")
    println(yv)
end

Set parameter Presolve to value 0
Set parameter Cuts to value 0
Set parameter Heuristics to value 0
Set parameter Threads to value 1
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]true1.0
1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [40, 41, 42, 43, 44, 45, 46, 47, 48, 49  …  771, 772, 773, 774, 775, 776, 777, 778, 779, 780]
And data, a 741-element Vector{Float64}:
 0.0
 0.0
 0.5
 0.5
 0.0
 0.5
 0.0
 0.5
 0.0
 0.5
 0.0
 0.5
 0.0
 0.5
 0.5
 0.5
 0.0
 0.5
 0.5
 0.5
 0.0
 0.5
 0.0
 0.0
 0.5
 0.5
 0.5
 0.5
 0.5
 0.0
 0.0
 0.0
 0.5
 0.5
 0.0
 0.0
 0.5
 0.5
 0.0
 0.5
 0.5
 0.0
 0.5
 0.0
 0.0
 0.5
 0.5
 0.0
 0.5
 0.0
 0.5
 0.5
 0.5
 0.0
 0.5
 0.5
 0.5
 0.0
 0.5
 0.0
 0.5
 0.5
 0.5
 0.0
 0.5
 0.5
 0.0
 0.0
 0.0
 0.5
 0.5
 0.0
 0.0
 0.5
 0.5
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0

LoadError: type String has no field format